In [ ]:
from strictly_typed_pandas import DataSet

class SchemaA:
    id: int
    a: int

class SchemaB:
    id: int
    b: int

class SchemaAB(SchemaA, SchemaB):
    pass


df_a = DataSet[SchemaA]({'id': [1], 'a': [1]})
df_b = DataSet[SchemaB]({'id': [1], 'b': [1]})

df = df_a.merge(df_b)
df

In [ ]:
from typing import get_type_hints

get_type_hints(df._schema)

In [ ]:
import pandas as pd
from strictly_typed_pandas import IndexedDataSet

class IndexSchemaA:
    id: int

class IndexSchemaB:
    id: int

class SchemaA:
    a: int

class SchemaB:
    b: int

class SchemaAB(SchemaA, SchemaB):
    pass

df_a = (
    pd.DataFrame(
        {
            "id": [1, 2, 3],
            "a": [1, 2, 3]
        }
    )
    .set_index(["id"])
    .pipe(IndexedDataSet[IndexSchemaA, SchemaA])
)
df_b = (
    pd.DataFrame(
        {
            "id": [1, 2, 3],
            "b": [1, 2, 3]
        }
    )
    .set_index(["id"])
    .pipe(IndexedDataSet[IndexSchemaB, SchemaB])
)

df = df_a.join(df_b)
df

In [ ]:
get_type_hints(df._schema)

In [ ]:
get_type_hints(df._index_schema)

In [5]:
import pandas as pd
import sys
from typing import TypeVar, Any, Optional
import typeguard
from typeguard import typechecked, _CallMemo, find_function, check_type
from strictly_typed_pandas import DataSet
from strictly_typed_pandas.core.join import Join

def check_argument_types(memo: Optional[_CallMemo] = None) -> bool:
    """
    Check that the argument values match the annotated types.

    Unless both ``args`` and ``kwargs`` are provided, the information will be retrieved from
    the previous stack frame (ie. from the function that called this).

    :return: ``True``
    :raises TypeError: if there is an argument type mismatch

    """
    if memo is None:
        # faster than inspect.currentframe(), but not officially
        # supported in all python implementations
        frame = sys._getframe(1)

        try:
            func = find_function(frame)
        except LookupError:
            return True  # This can happen with the Pydev/PyCharm debugger extension installed

        memo = _CallMemo(func, frame.f_locals)

    for argname, expected_type in memo.type_hints.items():
        if argname != 'return' and argname in memo.arguments:
            value = memo.arguments[argname]
            description = 'argument "{}"'.format(argname)
            try:
                check_type(description, value, expected_type, memo)
            except TypeError:
                raise TypeError()

    return True

typeguard.check_argument_types = check_argument_types

T = TypeVar('T')
class Schema:
    b: pd.PeriodDtype(freq="D")
class Schema2:
    b: int

@typechecked
def foo(a: DataSet[Schema]) -> DataSet[Any]:
    return a

foo(DataSet[Schema2]())

NameError: name 'get_type_hints' is not defined

In [10]:
from typing import get_type_hints
import numpy as np

class Schema:
    a: pd.DatetimeTZDtype(tz="UTC")  # type: ignore  # noqa: F821
    b: pd.CategoricalDtype
    c: pd.PeriodDtype(freq="D")  # type: ignore  # noqa: F821
    d: pd.SparseDtype(dtype=np.int64)  # type: ignore
    e: pd.IntervalDtype
    f: pd.Int64Dtype
    h: pd.BooleanDtype

get_type_hints(Schema)

{'a': datetime64[ns, UTC],
 'b': pandas.core.dtypes.dtypes.CategoricalDtype,
 'c': period[D],
 'd': Sparse[int64, 0],
 'e': pandas.core.dtypes.dtypes.IntervalDtype,
 'f': pandas.core.arrays.integer.Int64Dtype,
 'h': pandas.core.arrays.boolean.BooleanDtype}